In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import sklearn
import pandas as pd
import numpy as np

In [3]:
import os
from os import listdir
import sys
import json

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [5]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


In [6]:
project_dir = "/xdisk/msurdeanu/alexeeva/data/annotations-related"

In [7]:
annotated_data = os.path.join(project_dir, "annotated_as_of_dec13_both_uganda_and_rice")
annotated_data

'/xdisk/msurdeanu/alexeeva/data/annotations-related/annotated_as_of_dec13_both_uganda_and_rice'

In [8]:
# df = pd.read_csv(os.path.join(annotated_data, "Subtask1-MainTask-double-annotation-prep-as-of-Nov2.tsv"), sep="\t")

In [9]:
# df.head()

In [10]:
# load annotated data
adf = pd.DataFrame()
for file in listdir(annotated_data):
    print(file)
    if file.endswith("tsv"):
        f_path = os.path.join(annotated_data, file)
        temp_df = pd.read_csv(f_path, sep='\t', usecols = ["paragraph", "mention text (just a few words around the trigger)","trigger","sentence","annotation: b (belief or attitude), n (not a belief and not an attitude)"]).dropna()
        print(len(temp_df))
        adf = pd.concat([adf, temp_df])

Subtask1-MainTask-double-annotation-prep-as-of-Nov2.tsv
1026


In [11]:
adf.head()

,sentence,trigger,mention text (just a few words around the trigger),"annotation: b (belief or attitude), n (not a belief and not an attitude)",paragraph
0,"Their willingness to use storage is , however ...",willingness,"Their willingness to use storage is, however, ...",b,"Exhibit 14: Average Quantity of Maize, Rice an..."
1,"They appreciated the participatory approach , ...",consider,they consider,b,"As part of our survey, most farmers recognize ..."
2,"Based on this study , 30 m of Shuttle Radar To...",considered,) data was considered in our CFD simulation us...,n,"In terms of simulation accuracy, the shapes of..."
3,Mechanical threshing was more popular in the D...,popular,was more popular in the Delta than in he Middl...,b,The Delta had less bird damage than the Middle...
4,"The authors hope that the thriving features , ...",hope,"The authors hope that the thriving features, c...",n,This monograph describes the genetic resources...


In [12]:
adf["sentence"] = [s.strip() for s in adf["sentence"]]
adf = adf.drop_duplicates(subset = ["sentence", "mention text (just a few words around the trigger)"])

In [13]:
# len(df)

In [14]:
anns = adf["annotation: b (belief or attitude), n (not a belief and not an attitude)"]

In [15]:
list(anns).count("b")

377

In [16]:
# percentage of sentences annotated as beliefs
386/1026

0.3762183235867446

In [17]:
# annotated + sampled
df = adf
print("Annotated =", len(df))

Annotated = 1001


In [18]:
df['label'] = np.array([1 if x == "b" else 0 for x in df['annotation: b (belief or attitude), n (not a belief and not an attitude)']])

In [19]:
df.head()

,sentence,trigger,mention text (just a few words around the trigger),"annotation: b (belief or attitude), n (not a belief and not an attitude)",paragraph,label
0,"Their willingness to use storage is , however ...",willingness,"Their willingness to use storage is, however, ...",b,"Exhibit 14: Average Quantity of Maize, Rice an...",1
1,"They appreciated the participatory approach , ...",consider,they consider,b,"As part of our survey, most farmers recognize ...",1
2,"Based on this study , 30 m of Shuttle Radar To...",considered,) data was considered in our CFD simulation us...,n,"In terms of simulation accuracy, the shapes of...",0
3,Mechanical threshing was more popular in the D...,popular,was more popular in the Delta than in he Middl...,b,The Delta had less bird damage than the Middle...,1
4,"The authors hope that the thriving features , ...",hope,"The authors hope that the thriving features, c...",n,This monograph describes the genetic resources...,0


In [20]:
# from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# transformer_name = 'roberta-base'
transformer_name = "bert-base-cased"
# for cross validation, the model should be initialized inside the cv loop
# model = AutoModelForSequenceClassification.from_pretrained(transformer_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(transformer_name)

In [21]:
def tokenize(batch):
    return tokenizer(batch['sentence'], truncation=True)

In [22]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    report = metrics.classification_report(y_true, y_pred)
    print("report: \n", report)
    
    print("rep type: ", type(report))
    

    return {'f1':metrics.f1_score(y_true, y_pred)}

In [23]:


from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )



In [24]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=train_ds,
#     eval_dataset=eval_ds,
#     tokenizer=tokenizer,
#     )

In [25]:
from sklearn.model_selection import KFold

In [26]:
from transformers import AutoConfig


In [27]:
def get_sample_based_on_idx(data, indeces):
    return data.iloc[indeces, :].reset_index()

In [28]:
df.head(50)

,sentence,trigger,mention text (just a few words around the trigger),"annotation: b (belief or attitude), n (not a belief and not an attitude)",paragraph,label
0,"Their willingness to use storage is , however ...",willingness,"Their willingness to use storage is, however, ...",b,"Exhibit 14: Average Quantity of Maize, Rice an...",1
1,"They appreciated the participatory approach , ...",consider,they consider,b,"As part of our survey, most farmers recognize ...",1
2,"Based on this study , 30 m of Shuttle Radar To...",considered,) data was considered in our CFD simulation us...,n,"In terms of simulation accuracy, the shapes of...",0
3,Mechanical threshing was more popular in the D...,popular,was more popular in the Delta than in he Middl...,b,The Delta had less bird damage than the Middle...,1
4,"The authors hope that the thriving features , ...",hope,"The authors hope that the thriving features, c...",n,This monograph describes the genetic resources...,0
5,"Of these , the EAC , COMESA , and SADC are all...",considered,SADC are all considered significant regional i...,n,"Of these, the EAC, COMESA, and SADC are all co...",0
6,"Moreover , consumers already prefer local rice...",prefer,", consumers already prefer local rice in that ...",b,Since the introduction of improved rice variet...,1
7,You need to be especially careful with items t...,want,don’t want to inadvertently violate the EAR.,n,"Suppliers, consultants, and other third-partie...",0
8,"Needless to say , it is the reflection of the ...",say,"Needless to say, it is the reflection of the c...",n,"After PAPRIZ started its activity, it became c...",0
9,Government and political stability were long r...,perceived,is no longer perceived as a major constraint.,b,New actions are possible to address the multip...,1


In [29]:
len(df)

1001

In [30]:
set(df["label"])

{0, 1}

In [31]:
df["text"] = df["sentence"]

In [32]:
df.head()

,sentence,trigger,mention text (just a few words around the trigger),"annotation: b (belief or attitude), n (not a belief and not an attitude)",paragraph,label,text
0,"Their willingness to use storage is , however ...",willingness,"Their willingness to use storage is, however, ...",b,"Exhibit 14: Average Quantity of Maize, Rice an...",1,"Their willingness to use storage is , however ..."
1,"They appreciated the participatory approach , ...",consider,they consider,b,"As part of our survey, most farmers recognize ...",1,"They appreciated the participatory approach , ..."
2,"Based on this study , 30 m of Shuttle Radar To...",considered,) data was considered in our CFD simulation us...,n,"In terms of simulation accuracy, the shapes of...",0,"Based on this study , 30 m of Shuttle Radar To..."
3,Mechanical threshing was more popular in the D...,popular,was more popular in the Delta than in he Middl...,b,The Delta had less bird damage than the Middle...,1,Mechanical threshing was more popular in the D...
4,"The authors hope that the thriving features , ...",hope,"The authors hope that the thriving features, c...",n,This monograph describes the genetic resources...,0,"The authors hope that the thriving features , ..."


In [33]:
num_epochs = 20
batch_size = 8
weight_decay = 0.01
# model_name = f'{transformer_name}-sequence-classification'
training_args = TrainingArguments(
#     output_dir="./results",
#     log_level='error',
#     num_train_epochs=num_epochs,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     evaluation_strategy='epoch',
#     weight_decay=weight_decay,
#     save_total_limit = 1,
#       # report_to = "none",
#   #     logging_steps = 'epoch',
#     #     load_best_model_at_end=True,
#     metric_for_best_model="eval_f1",
#     load_best_model_at_end=True,
#     save_strategy = "epoch"
    output_dir="./results",
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1"
    )

In [34]:


fold = 0
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
for train_df_idx, eval_df_idx in kfold.split(df):
    print("FOLD: ", fold)
    new_df = pd.DataFrame()
    
    train_df = get_sample_based_on_idx(df, train_df_idx)
    print("LEN DF: ", len(train_df))
#     train_df['label'] = [int(item) for item in train_df["annotation: b (belief or attitude), n (not a belief and not an attitude)"]]
    print("done train df")
    eval_df = get_sample_based_on_idx(df, eval_df_idx)
#     eval_df["label"] = [int(item) for item in eval_df['annotation: b (belief or attitude), n (not a belief and not an attitude)']]
    print("done eval df")
    print("LEN EVAL: ", len(eval_df))
    print(eval_df.head())
    ds = DatasetDict()
    ds['train'] = Dataset.from_pandas(train_df)
    ds['validation'] = Dataset.from_pandas(eval_df)
    train_ds = ds['train'].map(
        tokenize, batched=True,
        remove_columns=['index', 'sentence', 'trigger', 'annotation: b (belief or attitude), n (not a belief and not an attitude)', 'paragraph'],
    )
    eval_ds = ds['validation'].map(
        tokenize,
        batched=True,
        remove_columns=['index', 'sentence', 'trigger', 'annotation: b (belief or attitude), n (not a belief and not an attitude)', 'paragraph'],
    )
    print(ds["validation"])

    config = AutoConfig.from_pretrained(
        transformer_name,
        num_labels=2,
    )

#     model = AutoModelForSequenceClassification.from_pretrained(transformer_name, num_labels=2)
#     tokenizer = AutoTokenizer.from_pretrained(transformer_name)
    model = (
        BertForSequenceClassification
        .from_pretrained(transformer_name, config=config)
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        tokenizer=tokenizer,
    )
    trainer.train()
    # after training, predict (will use best model?)
    preds = trainer.predict(eval_ds)
#     print(preds, "<<<")
    final_preds = [np.argmax(x) for x in preds.predictions]
#     print(final_preds, "<--")
#     print("labs: ", eval_df["label"])
    real_f1 = metrics.f1_score(final_preds, eval_df["label"])
    print("F-1: ", real_f1)
    count_f_n = 0
    count_f_p = 0
    for i, item in enumerate(final_preds):
        if not item == eval_ds["label"][i]:
            false_df = pd.DataFrame()
            false_df["sentence"] = [eval_df["sentence"][i]]
            false_df["real"] = [eval_df["label"][i]]
            false_df["predicted"] = [item]
            new_df = pd.concat([new_df, false_df])
#             print("NEW: \n", false_df.head())
            if item == 0:
                count_f_n += 1

            else:
                count_f_p += 1
#                 print(eval_ds["sentence"][i], " " , eval_ds["label"][i], " ", item, "\n")

    #     else:
    #         print(">>>", list(X_test)[i], " " , y_test_enc[i], " ", list(y_test)[i], " ", item, "\n")
    print(count_f_n)
    print(count_f_p)

    
#     print(new_df.head())
#     new_df.to_csv(os.path.join("/xdisk/msurdeanu/alexeeva/data/annotations-related/false-pos-neg", "false_annotations_" + str(fold) + ".tsv"), sep="\t")  
    fold += 1
    

FOLD:  0
LEN DF:  900
done train df
done eval df
LEN EVAL:  101
   index                                           sentence    trigger  \
0      6  Moreover , consumers already prefer local rice...     prefer   
1     17  Farmers may not need insurance as rice grown i...  perceived   
2     34  Crystal ( 2004 ) found that the world is facin...      found   
3     35  This study contributes by presenting and compa...  perceived   
4     41  While there are good reasons to believe that g...    believe   

  mention text (just a few words around the trigger)  \
0  , consumers already prefer local rice in that ...   
1  yields and was perceived to have lower risks t...   
2                               Crystal (2004) found   
3  econometrically estimated and perceived bird d...   
4  good reasons to believe that general equilibri...   

  annotation: b (belief or attitude), n (not a belief and not an attitude)  \
0                                                  b                        

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['index', 'sentence', 'trigger', 'mention text (just a few words around the trigger)', 'annotation: b (belief or attitude), n (not a belief and not an attitude)', 'paragraph', 'label', 'text'],
    num_rows: 101
})


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

...

Dataset({
    features: ['index', 'sentence', 'trigger', 'mention text (just a few words around the trigger)', 'annotation: b (belief or attitude), n (not a belief and not an attitude)', 'paragraph', 'label', 'text'],
    num_rows: 100
})


/home/u3/alexeeva/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,0.520840,0.439024
2,No log,0.410993,0.655738
3,No log,0.881910,0.655738
4,No log,0.911643,0.746667
5,0.309300,1.066156,0.666667
6,0.309300,1.407700,0.647887
7,0.309300,1.757258,0.615385
8,0.309300,1.815687,0.642857
9,0.040900,1.920841,0.641026
10,0.040900,1.733687,0.646154


report: 
               precision    recall  f1-score   support

           0       0.76      0.99      0.86        69
           1       0.90      0.29      0.44        31

    accuracy                           0.77       100
   macro avg       0.83      0.64      0.65       100
weighted avg       0.80      0.77      0.73       100

rep type:  <class 'str'>
report: 
               precision    recall  f1-score   support

           0       0.84      0.86      0.85        69
           1       0.67      0.65      0.66        31

    accuracy                           0.79       100
   macro avg       0.75      0.75      0.75       100
weighted avg       0.79      0.79      0.79       100

rep type:  <class 'str'>
report: 
               precision    recall  f1-score   support

           0       0.84      0.86      0.85        69
           1       0.67      0.65      0.66        31

    accuracy                           0.79       100
   macro avg       0.75      0.75      0.75     

report: 
               precision    recall  f1-score   support

           0       0.95      0.77      0.85        69
           1       0.64      0.90      0.75        31

    accuracy                           0.81       100
   macro avg       0.79      0.84      0.80       100
weighted avg       0.85      0.81      0.82       100

rep type:  <class 'str'>
F-1:  0.7466666666666666
3
16


In [35]:
# print(train_df["label"])

In [36]:
# torch.cuda.memory_summary(device=None, abbreviated=False)